In [1]:
import numpy as np
import pandas as pd


In [2]:
def load_df(path):
    train_df = pd.read_csv(path)
    X = train_df['id']
    y = train_df['landmark_id']
    
    return train_df, X, y

path = 'train_200.csv'

train_df, X, y = load_df(path)

In [3]:
NUM_CLASSES = train_df['landmark_id'].unique().shape[0]
NUM_CLASSES

1066

In [4]:
# map selected landmark ids to classes
# {landmark_id: class} 
# {995:0, 12345:1, ....}
landmarks = train_df['landmark_id'].unique()
landmark_to_idx = {}
i = 0
for k in landmarks:
    landmark_to_idx[k] = i
    i += 1

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

# split train & (validation + test)
split_rule = StratifiedShuffleSplit(n_splits=1, test_size=0.22, random_state=9)

for train_id, test_id in split_rule.split(X, y):
    X_train, X_val_test = X.iloc[train_id], X.iloc[test_id]
    y_train, y_val_test = y.iloc[train_id], y.iloc[test_id]

# split validation & test
split_rule2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=9)
for train_id, test_id in split_rule2.split(X_val_test, y_val_test):
    X_valid, X_test = X_val_test.iloc[train_id], X_val_test.iloc[test_id]
    y_valid, y_test = y_val_test.iloc[train_id], y_val_test.iloc[test_id]

In [6]:
print('X_train shape: ', X_train.shape)
print('y_train shape: ', y_train.shape)
print('X_valid shape: ', X_valid.shape)
print('y_valid shape: ', y_valid.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (630286,)
y_train shape:  (630286,)
X_valid shape:  (88887,)
y_valid shape:  (88887,)
X_test shape:  (88887,)
y_test shape:  (88887,)


In [7]:
valid_id_list = list(zip(list(X_valid), list(y_valid)))
train_id_list = list(zip(list(X_train), list(y_train)))

In [8]:
import os
import random
import shutil
import tarfile
import cv2
import numpy as np
#from keras.utils import Sequence
from tensorflow.python.keras.utils.data_utils import Sequence
#import keras



class DataGen(Sequence):
    def __init__(self, id_list, landmark_to_idx, batch_size=128, verbose=1):
        self.batch_size=batch_size
        self.id_list = id_list
        self.landmark_to_idx = landmark_to_idx


    def __getitem__(self, index):
        batch_id_list = random.sample(self.id_list, self.batch_size)
        landmark_to_idx = self.landmark_to_idx
        #num_classes = self.num_classes
        
        output = []
        label_idx = []
        for ix, ids in enumerate(batch_id_list):
            img_id = ids[0]
            ldmk_id = ids[1]
            path = 'train/'+str(ldmk_id)+'/'+img_id+'.jpg'
            try: 
                im = cv2.imread(path)
                im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
                if im.size != 0:
                    output.append(im)
                    ldmk_idx = landmark_to_idx[ldmk_id]
                    label_idx.append(ldmk_idx)
            except:
                continue
        
        x = np.array(output)
        y = np.zeros((len(output), NUM_CLASSES))
        for i in range(len(label_idx)):
            y[i,label_idx[i]] = 1.
        
        return x,y
            
    def on_epoch_end(self):
        return

    def __len__(self):
        #return len(valid_urls_list) // self.batch_size
        return 10

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passi

In [9]:
validation_generator = DataGen(valid_id_list, landmark_to_idx)
training_generator = DataGen(train_id_list, landmark_to_idx)

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input, Flatten
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

2.0.0-alpha0


In [11]:
from tensorflow.keras.applications.vgg16 import VGG16
vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=(128, 128, 3))

In [36]:
for layer in vgg16.layers[:5]:
    layer.trainable = False

x = vgg16.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(vgg16.input, predictions)
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['categorical_accuracy'])

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

In [37]:
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True, 
                    epochs=80,
                    workers=8,
                    verbose=1)

Epoch 1/80
10/10 [==============================] - 27s 3s/step - loss: 36.6533 - categorical_accuracy: 0.0047 - val_loss: 8.2983 - val_categorical_accuracy: 0.0016
Epoch 2/80
10/10 [==============================] - 19s 2s/step - loss: 7.4318 - categorical_accuracy: 0.0266 - val_loss: 7.1756 - val_categorical_accuracy: 0.0016
Epoch 3/80
10/10 [==============================] - 19s 2s/step - loss: 6.9716 - categorical_accuracy: 0.0047 - val_loss: 6.9159 - val_categorical_accuracy: 0.0094
Epoch 4/80
10/10 [==============================] - 19s 2s/step - loss: 6.9092 - categorical_accuracy: 0.0117 - val_loss: 6.8484 - val_categorical_accuracy: 0.0109
Epoch 5/80
10/10 [==============================] - 19s 2s/step - loss: 6.8029 - categorical_accuracy: 0.0078 - val_loss: 6.7597 - val_categorical_accuracy: 0.0086
Epoch 6/80
10/10 [==============================] - 18s 2s/step - loss: 6.6415 - categorical_accuracy: 0.0117 - val_loss: 6.5115 - val_categorical_accuracy: 0.0367
Epoch 7/80
10/1

10/10 [==============================] - 13s 1s/step - loss: 6.1567 - categorical_accuracy: 0.0594 - val_loss: 6.0601 - val_categorical_accuracy: 0.0672
Epoch 51/80
10/10 [==============================] - 13s 1s/step - loss: 5.9395 - categorical_accuracy: 0.0812 - val_loss: 5.8855 - val_categorical_accuracy: 0.0680
Epoch 52/80
10/10 [==============================] - 13s 1s/step - loss: 6.0214 - categorical_accuracy: 0.0648 - val_loss: 5.9076 - val_categorical_accuracy: 0.0773
Epoch 53/80
10/10 [==============================] - 13s 1s/step - loss: 6.0303 - categorical_accuracy: 0.0523 - val_loss: 5.9408 - val_categorical_accuracy: 0.0656
Epoch 54/80
10/10 [==============================] - 13s 1s/step - loss: 6.0392 - categorical_accuracy: 0.0758 - val_loss: 6.0397 - val_categorical_accuracy: 0.0641
Epoch 55/80
10/10 [==============================] - 13s 1s/step - loss: 5.9364 - categorical_accuracy: 0.0773 - val_loss: 5.9256 - val_categorical_accuracy: 0.0812
Epoch 56/80
10/10 [===

error: OpenCV(4.1.1) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [12]:
from tensorflow.keras.applications.resnet50 import ResNet50

resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3))

/opt/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [13]:
for layer in resnet.layers[-5:]:
    layer.trainable = False
    
out = Flatten()(resnet.output)
out = Dense(NUM_CLASSES, activation='softmax')(out)
model2 = Model(resnet.input, out)
model2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 64, 64, 64)   256         conv1[0][0]                      
______________________________________________________________________________________________

In [14]:
opt = Adam(0.0001)
model2.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['categorical_accuracy'])
model2.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs=80,
                    use_multiprocessing=True,
                    workers=8,
                    verbose=1)

Epoch 1/80
10/10 [==============================] - 46s 5s/step - loss: 8.0303 - categorical_accuracy: 0.0586 - val_loss: 9.0336 - val_categorical_accuracy: 0.0789
Epoch 2/80
10/10 [==============================] - 9s 885ms/step - loss: 6.4537 - categorical_accuracy: 0.1023 - val_loss: 8.8097 - val_categorical_accuracy: 0.0984
Epoch 3/80
10/10 [==============================] - 10s 973ms/step - loss: 5.9400 - categorical_accuracy: 0.1523 - val_loss: 9.1744 - val_categorical_accuracy: 0.1242
Epoch 4/80
10/10 [==============================] - 10s 966ms/step - loss: 5.7015 - categorical_accuracy: 0.1734 - val_loss: 8.2949 - val_categorical_accuracy: 0.1391
Epoch 5/80
10/10 [==============================] - 9s 880ms/step - loss: 5.6734 - categorical_accuracy: 0.1641 - val_loss: 7.8512 - val_categorical_accuracy: 0.1586
Epoch 6/80
10/10 [==============================] - 10s 1s/step - loss: 5.1912 - categorical_accuracy: 0.2109 - val_loss: 7.8379 - val_categorical_accuracy: 0.1625
Epoch 